In [1]:
import pandas as pd
import requests
import json
import datetime
import csv
import matplotlib.pyplot as plt
import time

In [3]:
TOKEN = "ee3oyaYzlZrYuCC0611Oezn0N"
base_url = "https://data.cityofnewyork.us/resource/t4f2-8md7.json"
headers = {"X-App-Token": TOKEN}

limit = 1000    
offset = 0
all_records = []

while True:
    url = f"{base_url}?$limit={limit}&$offset={offset}"
    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("Request failed, retrying in 2 seconds...", e)
        time.sleep(2)
        continue

    batch = response.json()

    if not batch:
        print("No more data returned. Stopping.")
        break

    all_records.extend(batch)
    print(f"Fetched {len(batch)} rows (offset={offset})")

    if len(batch) < limit:
        break

    offset += limit
    time.sleep(0.2)  # polite rate-limit protection

# Convert to pandas DataFrame
bus = pd.DataFrame(all_records)

print("Done! Total rows:", len(bus))
print(bus.head())


Fetched 1000 rows (offset=0)
Fetched 1000 rows (offset=1000)
Fetched 1000 rows (offset=2000)
Fetched 381 rows (offset=3000)
Done! Total rows: 3381
                                            the_geom boro_code boro_name  \
0  {'type': 'Point', 'coordinates': [-73.947831, ...         3  Brooklyn   
1  {'type': 'Point', 'coordinates': [-73.945162, ...         3  Brooklyn   
2  {'type': 'Point', 'coordinates': [-73.945783, ...         3  Brooklyn   
3  {'type': 'Point', 'coordinates': [-73.940836, ...         3  Brooklyn   
4  {'type': 'Point', 'coordinates': [-73.969795, ...         3  Brooklyn   

  boro_cd coun_dist assem_dist st_sen_dist cong_dist shelter_id corner  \
0     301        34         53          18         7     BR0003     SE   
1     301        34         50          18         7     BR0014     NE   
2     301        34         53          18         7     BR0026     NE   
3     301        34         50          18         7     BR0028     SW   
4     302        35       

In [4]:
columns_to_keep = ["boro_code", "boro_name", "longitude", "latitude", "shelter_id", "ntaname"]


In [5]:
bus = bus[columns_to_keep]

In [6]:
bus.head()

,boro_code,boro_name,longitude,latitude,shelter_id,ntaname
0,3,Brooklyn,-73.947831,40.706812,BR0003,East Williamsburg
1,3,Brooklyn,-73.945162,40.719097,BR0014,East Williamsburg
2,3,Brooklyn,-73.945783,40.702976,BR0026,East Williamsburg
3,3,Brooklyn,-73.940836,40.720195,BR0028,East Williamsburg
4,3,Brooklyn,-73.969795,40.69344,BR0040,Fort Greene


In [7]:
bus.ntaname.nunique()

231

In [8]:
shelter_count = bus.groupby('ntaname')['shelter_id'].count()
print(shelter_count)

ntaname
Allerton                                  10
Alley Pond Park                            4
Annadale-Huguenot-Prince's Bay-Woodrow     6
Arden Heights-Rossville                    4
Astoria (Central)                         17
                                          ..
Windsor Terrace-South Slope               10
Woodhaven                                 17
Woodlawn Cemetery                          4
Woodside                                  16
Yankee Stadium-Macombs Dam Park            1
Name: shelter_id, Length: 231, dtype: int64
